### Description:

AES-128 algorithm with a hardcoded key is available as service.

To perform encryption, the binary requires 16-bytes plaintext (in hex):

```
    ../wbcenc --ptext="010203040506070809000a0b0c0d0e1f"
```    
The result is printed to console:
```
    5FB9620C3E645F5487FCC688C07794C2 05070305060605030204010405030503
```    
where:
```
    5FB9620C3E645F5487FCC688C07794C2 is a ciphertext
    
    05070305060605030204010405030503 is a trace (information left by a programmer)
```
### Tips:

A 'trace' is a side-channel information left by a programmer to debug the implementation.

In this example a trace consists of Hamming weights for a 10th round input state bytes of the AES-128 algorithm, i.e., a State before the Sbox operation in the last round.

### Task:

Your task is to find the Master key (round key 0) embedded into the binary. 

The master key is in the form of HEIG{XXXXXXXXXX}, where X is an ASCII printable symbol.

### Leakage illustration

<img src="support/Slide.png">

In [6]:
import copy

import numpy as np

import sca_training

init = ["5FB9620C3E645F5487FCC688C07794C2", "05070305060605030204010405030503"]
init = [sca_training.hex_int(init) for init in init]
init[1] = sca_training.shift_rows(init[1])

cipher = ["746B3D0E1123FE61B3D9C910F24DA3C8","E9A53E0282388C091BE3A1FE2C179DEF","A3084819FF63B722AA644E05A749B4B1","5F6F1B977D84EBF044F52BD435196CBB","BA23D8046B55AD685CB0FF94854D427F","9F122BCAFB9048EEDC86A414E631AF90","A48BCFAD3CFFAB4C9A72CA0911DCCBCA","BE10478CB0A265CCCE7A5B640A33158B","0436DAAC1D36A67063F6AEC5DBCDC0D5","1B892D0D850C6DBE3B117DB1EA1D83A8","4B5CCBB1B73108C95F914931731D281B","EAE906F4224A8020D2FABC54C7197F4F","31A80C049F2540268D76A7498360474C","72BE5499531FD01927AD8D4DE5537F32","94935DFE0F612AD22C85D42E0F01E1B7",]
cipher = [sca_training.hex_int(cipher) for cipher in cipher]
trace = ["03030301050303030105020305060401","04020104020406020404060407020505","03030304030205030306040404050505","05030304040405040301050303070406","06030203050304030702040406050103","02040405050404050204070503040405","04060404050303060404020306050302","04050404060403050403030505040505","04040303040202040203040405040104","07040503020306000406050506030104","02040305020406040405050404070705","03030204030304050302040302020301","03030405060305040602030203050503","04030303050404050403020205050403","06030402060203040404010604040405",]
trace = [sca_training.shift_rows(sca_training.hex_int(trace)) for trace in trace]
assert(len(cipher) == len(trace))
candidates = [set() for _ in range(16)]

# Init candidates
for i in range(16): # Pour chaque position de la MK
    for candidate in range(256): # Pour chaque valeur possible du byte
        pos_trace = init[1][i]
        pos_cipher = init[0][i]
        mk = np.bitwise_xor(candidate, pos_cipher)
        mk = sca_training.invSbox[mk]
        if sca_training.HW_uint8[mk] == pos_trace:
            candidates[i].add(candidate)

# Filter candidates
for i in range(16):
    copy_candidates = copy.deepcopy(candidates[i])
    for j in range(len(cipher)):
        if len(candidates[i]) == 1:
            continue
        for candidate in copy_candidates:
            if len(candidates[i]) == 1:
                break
            pos_trace = trace[j][i]
            pos_cipher = cipher[j][i]
            mk = np.bitwise_xor(candidate, pos_cipher)
            mk = sca_training.invSbox[mk]
            if sca_training.HW_uint8[mk] != pos_trace and candidate in candidates[i]:
                candidates[i].remove(candidate)

print(candidates)
k10 = np.array([c.pop() for c in candidates])
print(k10)
mk = sca_training.k10_to_mk(k10)
print(mk)
print(sca_training.mk_to_ascii(mk))

[{79}, {204}, {144}, {62}, {229}, {78}, {218}, {104}, {131}, {42}, {108}, {210}, {68}, {35}, {9}, {30}]
[ 79 204 144  62 229  78 218 104 131  42 108 210  68  35   9  30]
[ 72  69  73  71 123  72  97 109 109 105 110 103  72  87  50 125]
HEIG{HammingHW2}


### ATTACK CODE
Here you need to implement your attack which gives you the last round key.

Once the last round key is found - you need to compute the master key: this can be done with the function get_master_key()

### Programming tips
#### Numpy code
* np.arange(256).astype(np.uint8)
* np.bitwise_xor(m1, m2)
* np.where()

#### Code prepared for training
* sca_training.invSbox[sbox_out]
* sca_training.shift_rows(trace)

#### Various cod
* ''.join('{:02x}'.format(c) for c in int_array)